In [1]:
import pandas as pd
from shapely import Polygon, Point, buffer, LineString, union_all, normalize

In [2]:
df_links = pd.read_json("links.json")

In [16]:
def parse_linestr(linestring):
    coord_list = linestring[11:][:-1].split(",")
    coord_list = [x.strip() for x in coord_list]
    point_list_out = []
    longitudes = []
    latitudes = []
    for coord in coord_list:
        lon, lat = coord.split(" ")
        longitudes.append(float(lon))
        latitudes.append(float(lat))
        point_list_out.append((lon, lat))

    return point_list_out

def create_buffer(polyline, band_width=0.0001):
    ls = LineString(polyline)
    buffer_polygon = buffer(ls, distance=band_width, cap_style="flat")
    return buffer_polygon
    # if hasattr(buffer_polygon.boundary, 'geoms'):
    #     buffer_polygon = buffer_polygon.boundary.geoms[0]
    #     return Polygon(buffer_polygon.coords)

    # return Polygon(buffer_polygon.boundary.coords)

df_routes = pd.read_csv("final_result1.csv", index_col=0)
df_routes['polyline'] = df_routes['polyline'].apply(parse_linestr)
df_routes['buffer_polygon'] = df_routes['polyline'].apply(create_buffer)

In [17]:
def get_links_within_polygon(polygon: Polygon):
    filters = df_links['polyline'].apply(LineString).apply(polygon.intersects)
    return df_links[filters]['link_id'].values

In [18]:
from tqdm import tqdm
tqdm.pandas()

df_routes['links_within_buffer'] = df_routes['buffer_polygon'].progress_apply(get_links_within_polygon).progress_apply(lambda x: ','.join(x))

100%|██████████████████████████████████████████████████████████| 19817/19817 [00:00<00:00, 649008.53it/s]


In [19]:
df_routes.to_csv("pt_routes_with_adjacent_links.csv")

In [13]:
df_o = df_routes[df_routes.route_number == "3"].copy()
df_o['links_within_buffer'] = df_o['buffer_polygon'].apply(get_links_within_polygon).apply(lambda x: ','.join(x))
df_o

,trip_id,route_number,route_name,route_type,from_stop,from_stop_name,from_stop_lon,from_stop_lat,to_stop,to_stop_name,to_stop_lon,to_stop_lat,polyline,buffer_polygon,links_within_buffer
12773,1.T0.3-3-mjp-10.1.H,3,Melbourne University - East Malvern,Tram,18208,7-Flagstaff Railway Station/William St (Melbou...,144.956799,-37.812461,18210,5-Bourke St/William St (Melbourne City),144.958160,-37.815440,"[(144.95676486141, -37.8124692039324), (144.95...",POLYGON ((144.95819370306808 -37.8154164156341...,"53ef018e-748f-4f09-852a-954efaa70bd4,ce1c4520-..."
12774,1.T0.3-3-mjp-10.1.H,3,Melbourne University - East Malvern,Tram,18210,5-Bourke St/William St (Melbourne City),144.958160,-37.815440,18211,4-Collins St/William St (Melbourne City),144.959092,-37.817457,"[(144.95810243037, -37.8154572726358), (144.95...",POLYGON ((144.95830018206723 -37.8156417043421...,"fd6be3bb-bccb-4de0-a7ad-9218885b2607,bb214d2c-..."
12775,1.T0.3-3-mjp-10.1.H,3,Melbourne University - East Malvern,Tram,18211,4-Collins St/William St (Melbourne City),144.959092,-37.817457,18214,1-Flinders St/Queens Bridge St (Melbourne City),144.961240,-37.819513,"[(144.959034923859, -37.8174830599149), (144.9...",POLYGON ((144.9596256206492 -37.81851677921624...,"bd48d800-472a-47e5-b757-048b4d9f559c,531a6c6d-..."
12776,1.T0.3-3-mjp-10.1.H,3,Melbourne University - East Malvern,Tram,18214,1-Flinders St/Queens Bridge St (Melbourne City),144.961240,-37.819513,18215,115-Casino/Southbank/Queens Bridge St (Southbank),144.961225,-37.822117,"[(144.961193791368, -37.8195300818646), (144.9...",POLYGON ((144.96129600077802 -37.8195169459465...,"9de9360a-99b6-410f-8e01-c0107e60a7e5,f3e7cb55-..."
12777,1.T0.3-3-mjp-10.1.H,3,Melbourne University - East Malvern,Tram,18215,115-Casino/Southbank/Queens Bridge St (Southbank),144.961225,-37.822117,22612,116-City Rd/Kings Way (Southbank),144.960803,-37.826300,"[(144.961191030602, -37.8221073936229), (144.9...",POLYGON ((144.96032113723672 -37.8251040525936...,"86f7c8a1-e615-4e98-87dc-50d16ca0455b,fc894702-..."
12778,1.T0.3-3-mjp-10.1.H,3,Melbourne University - East Malvern,Tram,22612,116-City Rd/Kings Way (Southbank),144.960803,-37.826300,18217,117-York St/Kings Way (South Melbourne),144.964230,-37.829766,"[(144.960756795944, -37.8263263802983), (144.9...",POLYGON ((144.96125914645316 -37.8267056194477...,"86f7c8a1-e615-4e98-87dc-50d16ca0455b,19f89745-..."
12779,1.T0.3-3-mjp-10.1.H,3,Melbourne University - East Malvern,Tram,18217,117-York St/Kings Way (South Melbourne),144.964230,-37.829766,18218,118-Sturt St/Kings Way (Southbank),144.965004,-37.830536,"[(144.964183304674, -37.8298011294706), (144.9...",POLYGON ((144.96502817917073 -37.8305100457357...,
12780,1.T0.3-3-mjp-10.1.H,3,Melbourne University - East Malvern,Tram,18218,118-Sturt St/Kings Way (Southbank),144.965004,-37.830536,18219,119-Wells St/Park St (South Melbourne),144.969816,-37.832800,"[(144.964957220273, -37.8305805073198), (144.9...",POLYGON ((144.96779425621003 -37.8333280408427...,"6581b894-8a12-4c25-8abd-a154179f140d,d17efc90-..."


In [15]:
df_o.links_within_buffer.values

array(['53ef018e-748f-4f09-852a-954efaa70bd4,ce1c4520-f8cf-49b8-a631-08954199e76a,4ed82198-6a3c-424a-b71a-fcb5d165ca6e,9d42929b-ffba-493d-b297-92c73bb46735,ba04f462-05b5-4c00-8fcb-c4f0ae36c3f4,817782f6-9b27-458a-bb13-34c3c2a09002,a97aa396-0a5e-414f-8641-3b1dc0944a0b,4a2f6bfa-88f2-43f7-8497-12ebe715d865',
       'fd6be3bb-bccb-4de0-a7ad-9218885b2607,bb214d2c-3963-466d-82c6-0edc5b356f75,4a2f6bfa-88f2-43f7-8497-12ebe715d865,22ffb574-c1aa-4a13-9cf8-bb21037ff99a',
       'bd48d800-472a-47e5-b757-048b4d9f559c,531a6c6d-ff8d-4a1f-a2bc-a2cf3715acc9,28434f90-39c3-4b6c-9831-0a3efb89a5d9,fd6be3bb-bccb-4de0-a7ad-9218885b2607,9b684a26-3247-4a05-bd1e-0e9953723096,6f99a937-b8f3-45d3-8f37-7eabc3da491d,5c56b585-b68c-4f04-9f3f-7e2173b6cd1d',
       '9de9360a-99b6-410f-8e01-c0107e60a7e5,f3e7cb55-0da7-44e7-b62f-589550b6d239,728e2f50-643c-488b-acb5-1987800d2e05,fc894702-68bb-4b30-b1f1-a695c73ab362,d304f22c-8441-476f-9d55-3c95171e7054,ef32ee45-669f-449e-abcb-c0b05ca352fa,338dc9a0-8fe2-456d-9ef3-9be31359f7cb'